In [ ]:
import numpy as np

from gym import Env
from gym.spaces import Discrete
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import warnings

warnings.filterwarnings("ignore")

X = [720, 1440, 2160, 2880, 3600, 4320, 5040, 5760]
Y_packet_delivery_ratio = []
Y_avg_end_to_end_delay = []
Y_routing_overhead_ratio = []

for x in X:

    NEIGHBOUR_DISTANCE = 250
    NUM_VEHICLES = 150
    DEST_ID = 9
    MAX_TIME = x
    GRID_X = 2000
    GRID_Y = 2000
    AVG_SPEED_X = 10
    AVG_SPEED_Y = 10
    AVG_SPEED_JITTER_X = 2
    AVG_SPEED_JITTER_Y = 2
    LOOK_BACK = 4
    ACCEPTABLE_TIME = 60
    SPEED_OF_LIGHT = 300000000
    TRANSMISSION_RATE = 2*1024*1024
    MAX_NEIGHBOURS = 5

    class Vehicle():
        def __init__(self, id, resource_util, latitude, longitude, num_neighbours, neighbours):
            self.id = id
            self.resource_util = resource_util
            self.latitude = latitude
            self.longitude = longitude
            self.num_neighbours = num_neighbours
            self.neighbours = neighbours

        def __str__(self):
            return f"Vehicle(id={self.id}, resource_util={self.resource_util}, latitude={self.latitude}, longitude={self.longitude}, num_neighbours={self.num_neighbours}, neighbours={self.neighbours})"

    class MapEnv(Env):
        def __init__(self, num_vehicles, vehicles, init_car_id, dest_id, init_reward_table, init_distance_table, init_sf_table, init_cf_table, init_rf_table, init_time, total_time, R, a1, a2, a3, b1, b2, b3, max_neighbours):
            self.action_space = Discrete(num_vehicles)
            self.state_space = Discrete(num_vehicles)
            self.num_vehicles = num_vehicles
            self.vehicles = vehicles
            self.state = init_car_id
            self.dest_id = dest_id
            self.time = init_time
            self.total_time = total_time
            self.reward_table = init_reward_table
            self.distance_table = init_distance_table
            self.sf_table = init_sf_table
            self.cf_table = init_cf_table
            self.rf_table = init_rf_table
            self.R = R
            self.a1 = a1
            self.a2 = a2
            self.a3 = a3
            self.b1 = b1
            self.b2 = b2
            self.b3 = b3
            self.max_neighbours = max_neighbours
        
        def step(self, action):
            reward = self.reward_table[self.state][action]
            self.state = action
            self.time += 1 
            if self.time==self.total_time or self.state==self.dest_id:
                done = True
            else:
                done = False
            return self.state, reward, done, {}
        
        def reset(self, init_car_id, dest_id, init_time):
            self.state = init_car_id
            self.dest_id = dest_id
            self.time = init_time
            return self.state
        
        def calc_distance(self, x1, x2, y1, y2):
            return np.sqrt(np.power(x1-x2, 2)+np.power(y1-y2, 2))
        
        def get_sf(self, curr_id, nei_id):
            old_dist = self.distance_table[curr_id][nei_id]
            new_dist = self.calc_distance(self.vehicles[curr_id].latitude, self.vehicles[nei_id].latitude, self.vehicles[curr_id].longitude, self.vehicles[nei_id].longitude)
            self.distance_table[curr_id][nei_id] = new_dist
            old_sf = self.sf_table[curr_id][nei_id]
            new_sf = 0
            if np.abs(old_dist-new_dist) <= ((AVG_SPEED_X+AVG_SPEED_Y)/2):
                new_sf = 1 - np.abs(old_dist-new_dist)/((AVG_SPEED_X+AVG_SPEED_Y)/2)
            self.sf_table[curr_id][nei_id] = new_sf
            if old_dist==0:
                return 0
            return self.a1*new_sf + (1-self.a1)*old_sf
        
        def get_cf(self, curr_id, nei_id):
            old_cf = self.cf_table[curr_id][nei_id]
            new_cf = self.vehicles[nei_id].num_neighbours/self.max_neighbours
            self.cf_table[curr_id][nei_id] = new_cf
            return self.a2*new_cf + (1-self.a2)*old_cf
        
        def get_rf(self, curr_id, nei_id):
            old_rf = self.rf_table[curr_id][nei_id]
            new_rf = 1-self.vehicles[nei_id].resource_util
            self.rf_table[curr_id][nei_id] = new_rf
            return self.a3*new_rf + (1-self.a3)*old_rf
        
        def update_vehicles(self, vehicles):
            self.vehicles = vehicles
        
        def update_reward_table(self, curr_id, nei_id):
            x = self.get_sf(curr_id, nei_id)
            y = self.get_cf(curr_id, nei_id)
            z = self.get_rf(curr_id, nei_id)
            reward = self.b1*x + self.b2*y + self.b3*z
            if nei_id==self.dest_id:
                self.reward_table[curr_id][nei_id] = 0
            else:
                if reward==0 or -1/reward<=-1000:
                    self.reward_table[curr_id][nei_id] = -1000
                else:
                    self.reward_table[curr_id][nei_id] = -1/reward

    def init_dataset():
        dataset = []
        speedx = []
        speedy = []
        for i in range(NUM_VEHICLES):
            vehicle = Vehicle(id=i, resource_util=np.random.random(), latitude=np.random.randint(low=0, high=GRID_Y), longitude=np.random.randint(low=0, high=GRID_X), num_neighbours=0, neighbours=[])
            dataset.append(vehicle)
        for i in range(NUM_VEHICLES):
            for j in range(NUM_VEHICLES):
                if i!=j and np.sqrt(np.power(dataset[i].latitude-dataset[j].latitude, 2)+np.power(dataset[i].longitude-dataset[j].longitude, 2)) <= NEIGHBOUR_DISTANCE:
                    dataset[i].num_neighbours += 1
                    dataset[i].neighbours.append(j)
        for i in range(NUM_VEHICLES):
            speedx.append(AVG_SPEED_X+np.random.randint(low=-AVG_SPEED_JITTER_X, high=AVG_SPEED_JITTER_X+1))
            speedy.append(AVG_SPEED_Y+np.random.randint(low=-AVG_SPEED_JITTER_Y, high=AVG_SPEED_JITTER_Y+1))
        
        return dataset, speedx, speedy

    # packets dataset
    class Packet:
        def __init__(self, id, bits, src_id, dest_id, at_id, init_time):
            self.id = id
            self.bits = bits
            self.src_id = src_id
            self.dest_id = dest_id
            self.at_id = at_id
            self.time_taken = init_time

    def Q_learning(env, init_vehicle_id, dest_id):
        alpha = 0.9
        gamma = 0.95
        epsilon = 0.0
        num_episodes = 200

        Q = np.zeros((env.state_space.n, env.action_space.n))

        for _ in range(num_episodes):
            state = env.reset(init_vehicle_id, dest_id, 0)
            done = False

            while not done:
                if np.random.rand() < epsilon:
                    action = env.action_space.sample()
                else:
                    action = np.argmax(Q[state])

                next_state, reward, done, _ = env.step(action)
                Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state, action])
                state = next_state
        return Q
    
    def get_df_table(env, vehicle_history_x, vehicle_history_y):
        vehicle_future_x = np.array([-1]*env.num_vehicles)
        vehicle_future_y = np.array([-1]*env.num_vehicles)

        def calc_distance(x1, x2, y1, y2):
            return np.sqrt(np.power(x1-x2, 2)+np.power(y1-y2, 2))

        def distance(first_id, second_id):
            return calc_distance(vehicle_future_x[first_id], vehicle_future_x[second_id], vehicle_future_y[first_id], vehicle_future_y[second_id])

        def angle(start_id, middle_id, end_id):
            a = calc_distance(vehicle_future_x[start_id], vehicle_future_x[middle_id], vehicle_future_y[start_id], vehicle_future_y[middle_id])
            b = calc_distance(vehicle_future_x[end_id], vehicle_future_x[middle_id], vehicle_future_y[end_id], vehicle_future_y[middle_id])
            c = calc_distance(vehicle_future_x[end_id], vehicle_future_x[start_id], vehicle_future_y[end_id], vehicle_future_y[start_id])
            if a==0 or b==0:
                return 1
            return (np.power(c, 2)-np.power(a, 2)-np.power(b, 2))/(2*a*b)

        if len(vehicle_history_x)==LOOK_BACK:
            for vehicle_id in range(env.num_vehicles):
                model = ARIMA(vehicle_history_x[:-1, vehicle_id], order=(1, 1, 0), enforce_stationarity=False)
                model_fit = model.fit()
                forecast_steps = 1
                vehicle_future_x[vehicle_id] = int(model_fit.forecast(steps=forecast_steps)[0])

                model = ARIMA(vehicle_history_y[:-1, vehicle_id], order=(1, 1, 0), enforce_stationarity=False)
                model_fit = model.fit()
                forecast_steps = 1
                vehicle_future_y[vehicle_id] = int(model_fit.forecast(steps=forecast_steps)[0])
        else:
            for vehicle_id in range(env.num_vehicles):
                vehicle_future_x[vehicle_id] = vehicle_history_x[-1, vehicle_id]
                vehicle_future_y[vehicle_id] = vehicle_history_y[-1, vehicle_id]
        
        df_table = np.zeros((env.state_space.n, env.action_space.n))

        for curr_id in range(env.state_space.n):
            for nei_id in env.vehicles[curr_id].neighbours:
                df_table[curr_id, nei_id] = angle(curr_id, nei_id, env.dest_id)*np.exp(-distance(nei_id, env.dest_id)/env.R)

        return df_table

    class Result:
        def __init__(self, is_done, q_learning_time, arima_time, transmission_time, info_packets, index):
            self.is_done = is_done
            self.q_learning_time = q_learning_time
            self.arima_time = arima_time
            self.transmission_time = transmission_time
            self.info_packets = info_packets
            self.index = index

        def __str__(self):
            return f"Result(is_done={self.is_done}, q_learning_time={self.q_learning_time}, arima_time={self.arima_time}, transmission_time={self.transmission_time}, index={self.index})"

    results = []

    NUM_TEST = 1

    for test_index in tqdm(range(NUM_TEST)):

        Q_LEARNING_TIME = 0
        ARIMA_TIME = 0
        TRANSMISSION_TIME = 0
        PACKET_TIME = 0
        INFO_PACKETS = 0
        Index = MAX_TIME

        def calc_distance(x1, x2, y1, y2):
            return np.sqrt(np.power(x1-x2, 2)+np.power(y1-y2, 2))
    
        def sigmoid(z):
            return 1/(1 + np.exp(-z))

        init_reward_table = np.array([[-1000 for i in range(NUM_VEHICLES)] for j in range(NUM_VEHICLES)])
        init_distance_table = np.array([[0 for i in range(NUM_VEHICLES)] for j in range(NUM_VEHICLES)])
        init_sf_table = np.array([[0 for i in range(NUM_VEHICLES)] for j in range(NUM_VEHICLES)])
        init_cf_table = np.array([[0 for i in range(NUM_VEHICLES)] for j in range(NUM_VEHICLES)])
        init_rf_table = np.array([[0 for i in range(NUM_VEHICLES)] for j in range(NUM_VEHICLES)])

        env = MapEnv(
            num_vehicles=NUM_VEHICLES,
            vehicles=[],
            init_car_id=0,
            dest_id=DEST_ID,
            init_reward_table=init_reward_table,
            init_distance_table=init_distance_table,
            init_sf_table=init_sf_table,
            init_cf_table=init_cf_table,
            init_rf_table=init_rf_table,
            init_time=0,
            total_time=ACCEPTABLE_TIME,
            R=NEIGHBOUR_DISTANCE,
            a1=0.8,
            a2=0.8,
            a3=0.8,
            b1=0.2,
            b2=0.5,
            b3=0.2,
            max_neighbours=MAX_NEIGHBOURS
        )

        packet = Packet(id=1232, bits=65536, src_id=0, dest_id=DEST_ID, at_id=0, init_time=0)

        prev_dataset, prev_speedx, prev_speedy = init_dataset()
        dataset = [prev_dataset]

        for time in range(MAX_TIME):
            next_dataset = []
            next_speedx = []
            next_speedy = []
            
            for i in range(NUM_VEHICLES):
                vehicle = Vehicle(id=i, resource_util=np.random.random(), latitude=prev_dataset[i].latitude+prev_speedx[i], longitude=prev_dataset[i].longitude+prev_speedy[i], num_neighbours=0, neighbours=[])
                next_dataset.append(vehicle)
            for i in range(NUM_VEHICLES):
                for j in range(NUM_VEHICLES):
                    if i!=j and np.sqrt(np.power(next_dataset[i].latitude-next_dataset[j].latitude, 2)+np.power(next_dataset[i].longitude-next_dataset[j].longitude, 2)) <= NEIGHBOUR_DISTANCE:
                        next_dataset[i].num_neighbours += 1
                        next_dataset[i].neighbours.append(j)
            for i in range(NUM_VEHICLES):
                next_speedx.append(AVG_SPEED_X+np.random.randint(low=-AVG_SPEED_JITTER_X, high=AVG_SPEED_JITTER_X+1))
                next_speedy.append(AVG_SPEED_Y+np.random.randint(low=-AVG_SPEED_JITTER_Y, high=AVG_SPEED_JITTER_Y+1))

            prev_dataset = next_dataset
            next_speedx = prev_speedx
            next_speedy = prev_speedy

            dataset.append(next_dataset)

        for time in range(MAX_TIME):
            vehicles = dataset[time]
            vehicle_history_x = np.array([[vehicle.longitude for vehicle in data] for data in dataset[max(0, time-LOOK_BACK):time+1]])
            vehicle_history_y = np.array([[vehicle.latitude for vehicle in data] for data in dataset[max(0, time-LOOK_BACK):time+1]])

            if packet.dest_id==packet.at_id:
                if packet.time_taken <= (ACCEPTABLE_TIME/60):
                    PACKET_TIME = packet.time_taken
                    Index = time
                break

            env.update_vehicles(vehicles=vehicles)
            INFO_PACKETS += vehicles[packet.at_id].num_neighbours/LOOK_BACK
            for curr_id in range(NUM_VEHICLES):
                for nei_id in env.vehicles[curr_id].neighbours:
                    env.update_reward_table(curr_id=curr_id, nei_id=nei_id)
                    
            # Q-learning starts
            qlearn_start_time = datetime.now()
            Q = Q_learning(env, packet.at_id, packet.dest_id)
            qlearn_end_time = datetime.now()
            Q_LEARNING_TIME += (qlearn_end_time - qlearn_start_time).total_seconds()
            # Q-learning ends

            # ARIMA starts
            arima_start_time = datetime.now()
            df_table = get_df_table(env, vehicle_history_x, vehicle_history_y)
            Q_dash = sigmoid(Q/1000)*sigmoid(df_table)
            arima_end_time = datetime.now()
            ARIMA_TIME += (arima_end_time - arima_start_time).total_seconds()
            # ARIMA ends

            best_action = [np.argmax(Q_dash[i]) for i in range(env.state_space.n)]
            travel_dist = calc_distance(env.vehicles[packet.at_id].latitude, env.vehicles[best_action[packet.at_id]].latitude, env.vehicles[packet.at_id].longitude, env.vehicles[best_action[packet.at_id]].longitude)
            
            packet.time_taken += (travel_dist/SPEED_OF_LIGHT)+(packet.bits/TRANSMISSION_RATE)
            packet.at_id = best_action[packet.at_id]

        if PACKET_TIME==0:
            results.append(Result(is_done=False, q_learning_time=-1, arima_time=-1, transmission_time=-1, info_packets=int(INFO_PACKETS), index=Index))
        else:
            results.append(Result(is_done=True, q_learning_time=Q_LEARNING_TIME*1000, arima_time=ARIMA_TIME*1000, transmission_time=PACKET_TIME*1000, info_packets=int(INFO_PACKETS), index=Index))

    PACKET_LOSS = 0
    AVG_Q_LEARNING_TIME = 0
    AVG_ARIMA_TIME = 0
    AVG_TRANSMISSION_TIME = 0
    AVG_INFO_PACKETS = 0

    for res in results:
        if not res.is_done:
            PACKET_LOSS += 1
        else:
            AVG_Q_LEARNING_TIME += res.q_learning_time
            AVG_ARIMA_TIME += res.arima_time
            AVG_TRANSMISSION_TIME += res.transmission_time
            AVG_INFO_PACKETS += res.info_packets

    Y_packet_delivery_ratio.append( round((NUM_TEST-PACKET_LOSS)/NUM_TEST, 3) )
    Y_routing_overhead_ratio.append( round(np.sqrt(AVG_INFO_PACKETS/NUM_TEST*MAX_NEIGHBOURS*MAX_NEIGHBOURS), 0) )
    Y_avg_end_to_end_delay.append( round(((AVG_Q_LEARNING_TIME+AVG_ARIMA_TIME)+AVG_TRANSMISSION_TIME)/NUM_TEST/2, 0) )

    # print(f"For num_vehicles -> {NUM_VEHICLES}, max_time -> {MAX_TIME}")
    print(f"For max_time -> {MAX_TIME}, num_vehicles -> {NUM_VEHICLES}")
    print(f"    packet_delivery_ratio : {Y_packet_delivery_ratio[-1]}")
    print(f"    avg_end_to_end_delay : {Y_avg_end_to_end_delay[-1]}")
    print(f"    routing_overhead_ratio : {Y_routing_overhead_ratio[-1]}")
    print(f"    AVG_Q_LEARNING_TIME : {round(AVG_Q_LEARNING_TIME/NUM_TEST, 0)}")
    print(f"    AVG_ARIMA_TIME : {round(AVG_ARIMA_TIME/NUM_TEST, 0)}")
    print(f"    AVG_TRANSMISSION_TIME : {round(AVG_TRANSMISSION_TIME/NUM_TEST, 0)}")
    print()